In [1]:
CATBOOST_VERSION = 1.0

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn import set_config
from catboost import CatBoostRegressor
from datetime import timedelta

In [3]:
X_train = pd.read_csv('X_train.csv', encoding='cp949').drop(columns='ID')
y_train = pd.read_csv('y_train.csv', encoding='cp949').Salary

X_test = pd.read_csv('X_test.csv', encoding='cp949')
test_id = X_test.ID
X_test = X_test.drop(columns='ID')

In [4]:
df=pd.concat([X_train, X_test])

In [5]:
df['근무형태'] = df['근무형태'].fillna('신입')
df['어학시험'] = df['어학시험'].fillna('없음')

In [6]:
df['연']=df['근무경력'].str.split(" ").str[0]
df['개월']=df['근무경력'].str.split(" ").str[-1]
df1=pd.concat([df.연,df.개월],axis=1)

for i in {'0개월','1개월','2개월','3개월','4개월','5개월','6개월','7개월','8개월','9개월','10개월','11개월'}:
    df1['연'].loc[df1['연'] == i] = '0년'

df1['A'] = df1['연'].str.rstrip("년")
df1['B'] = df1['개월'].str.rstrip("개월")

df1['연차를 개월로 환산'] = df1['A'].astype(dtype='int64')*12
df1['B'] = df1['B'].astype(dtype='int64')*1
df['근무경력'] = df1['연차를 개월로 환산']+df1['B']

df=df.drop(['연'],axis=1)
df=df.drop(['개월'],axis=1)

In [7]:
pd.options.display.max_rows = 2686
df['대학전공'].value_counts()

경영학                                      1304
경영학과                                      749
경제학                                       533
컴퓨터공학                                     522
전자공학                                      384
경영                                        350
영어영문                                      347
기계공학                                      345
산업공학                                      341
전자공학과                                     339
회계학                                       330
행정학                                       328
기계공학과                                     323
법학                                        311
무역학                                       307
경제학과                                      283
산업디자인                                     270
영어영문학                                     269
컴퓨터공학과                                    258
무역학과                                      252
산업공학과                                     252
화학                                

In [69]:
for i , j in  zip(df['대학전공'].str.contains('서양'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'서양화',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('컴퓨터정보'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'컴퓨터정보공학',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('섬유예술'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'섬유예술학',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('영어교육'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'영어교육학',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('보험'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'금융보험학',inplace=True)

for i , j in  zip(df['대학전공'].str.contains('세무'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'세무회계학',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('천연섬유'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'천연섬유학',inplace=True)

for i , j in  zip(df['대학전공'].str.contains('섬유디자인'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'섬유디자인학',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('간호'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'간호학',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('전기전자제어'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'전기전자제어학',inplace=True)

for i , j in  zip(df['대학전공'].str.contains('전기전자정보'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'전기전자정보',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('전자통신'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'전자통신학',inplace=True)

for i , j in  zip(df['대학전공'].str.contains('전자정보통신'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'전자통신학',inplace=True)        

In [76]:
df['대학전공']=df['대학전공'].replace(
{
    '경영':'경영학',
    '경영학':'경영학',
    '경영학 전공':'경영학',
    '경영학과':'경영학',
    '경영학과 (주간)':'경영학',
    '경영학과(주간)':'경영학',
    '경영학부':'경영학',
    '경영학부 경영학':'경영학',
    '경영학부 경영학 전공':'경영학',
    '경영학사':'경영학',
    '경영정보':'경영정보학',
    '경영정보학과':'경영정보학',
    'MIS(경영정보시스템)':'경영정보학',
    '인터넷경영정보':'경영정보학',
    '경영정보(MIS)':'경영정보학',
    '경영정보과':'경영정보학',
    '경영정보시스템':'경영정보학',
    '경영정보과학부':'경영정보학',
    'MIS':'경영정보학',
    '경영(MIS)':'경영정보학',
    '정보처리(MIS)':'경영정보학',
    '경제학과':'경제학',
    '경제':'경제학',
    '경제학부':'경제학',
    '경제과':'경제학',
    '경제학사':'경제학',
    '컴퓨터공학과':'컴퓨터공학',
    '컴퓨터':'컴퓨터공학',
    '컴퓨터학과':'컴퓨터공학',
    '컴퓨터과학':'컴퓨터공학',
    '컴퓨터학':'컴퓨터공학',
    '컴퓨터과학과':'컴퓨터공학',
    '컴퓨터 공학':'컴퓨터공학',
    '컴퓨터공':'컴퓨터공학',
    '컴퓨터 공학과':'컴퓨터공학',
    '컴퓨터 과학과':'컴퓨터공학',
    '컴퓨터과':'컴퓨터공학',
    '컴퓨터공학전공':'컴퓨터공학',
    '통신컴퓨터공학부 컴퓨터전공':'컴퓨터공학',
    '컴퓨터 학부':'컴퓨터공학',
    '통신`컴퓨터공학분 컴퓨터전공':'컴퓨터공학',
    '전자공학과':'전자공학',
    '전자공학부':'전자공학',
    '기계전자공학부 전자 전공':'전자공학',
    '전기전자공학과':'전기전자공학',
    '전기전자공학부':'전기전자공학',
    '공대 전기전자공학과':'전기전자공학',
    '전기전자':'전기전자공학',
    '전자전기':'전기전자공학',
    '전기전자공':'전기전자공학',
    '전자전기공':'전기전자공학',
    '전자전기공':'전기전자공학',
    '전기전자 공학부':'전기전자공학',
    '전기,전자':'전기전자공학',
    '전자전기공학':'전기전자공학',
    '전기공학과':'전기공학',
    '전기':'전기공학',
    '전기공':'전기공학',
    '전기과':'전기공학',
    '전기 공학과':'전기공학',
    '전기공학부':'전기공학',
    '전기공학전공':'전기공학',
    '전기 공학':'전기공학',
    '전기학과':'전기공학',
    '전기공항':'전기공학',
    '전기공화과':'전기공학',
    '전기(전자)':'전기공학',
    '정보통신':'정보통신학',
    '정보통신공학과':'정보통신학',
    '정보통신공':'정보통신학',
    '정보통신학과':'정보통신학',
    '정보통신공학부':'정보통신학',
    '정보통신학':'정보통신학',
    '정보통신과':'정보통신학',
    '정보통신 공학과':'정보통신학',
    '정보통신학과(구 전자공학과)':'정보통신학',
    '정보통신전공':'정보통신학',
    '전기전자공학부 정보통신전공':'정보통신학',
    '정보통신 공학':'정보통신학',
    '정보통신공학사':'정보통신학',
    '컴퓨터정보공학부 정보통신공학전공':'정보통신학',
    '정보통신공학전공':'정보통신학',
    '영어영문':'영어영문학',
    '영어영문학과':'영어영문학',
    '영어학과':'영어영문학',
    '영어학':'영어영문학',
    '영어과':'영어영문학',
    '영어영문과':'영어영문학',
    '영어 영문학':'영어영문학',
    '영어 영문학과':'영어영문학',
    '영어 영문':'영어영문학',
    '영어영문학부':'영어영문학',
    '영어 영문과':'영어영문학',
    '영어영문학(야)':'영어영문학',
    '영어영문학사':'영어영문학',
    '영문영어':'영어영문학',
    '영어 전공':'영어영문학',
    '기계공학과':'기계공학',
    '기계공학부':'기계공학',
    '기계':'기계공학',
    '기계공':'기계공학',
    '기계과':'기계공학',
    '기계 공학과':'기계공학',
    '기계공악':'기계공학',
    '기계 공학':'기계공학',
    '기계공학 전공':'기계공학',
    '기계고학부':'기계공학',
    '기계공확과':'기계공학',
    '산업공학과':'산업공학',
    '산업공학전공':'산업공학',
    '회계학과':'회계학',
    '회계':'회계학',
    '회계헉':'회계학',
    '회계학과(야)':'회계학',
    '상경학부(회계전공)':'회계학',
    '경영학부(회계학)':'회계학',
    '회계확과':'회계학',
    '회계학과졸업':'회계학',
    '회계학':'회계학',
    '경영경제':'경영경제학',
    '경영경제 (주)':'경영경제학',
    '경영경제 전공':'경영경제학',
    '경영경제학과':'경영경제학',
    '행정학과':'행정학',
    '행정':'행정학',
    '행정학부':'행정학',
    '행정과':'행정학',
    '행정학전공':'행정학',
    '행정학과 졸업':'행정학',
    '법정대 행정학':'행정학',
    '법과대학 행정학':'행정학',
    '법학과':'법학',
    '법':'법학',
    '법학전공':'법학',
    '법(야)':'법학',
    '법정학부':'법학',
    '법과대학':'법학',
    '법학학사':'법학',
    '무역학과':'무역학',
    '무역':'무역학',
    '국제무역학과':'무역학',
    '무역학전공':'무역학',
    '경영학부 무역학':'무역학',
    '국제무역':'무역학',
    '무역과':'무역학',
    '경상학부 무역학':'무역학',
    '경상 무역학과':'무역학',
    '경상학부 무역학과':'무역학',
    '무역학과 졸업':'무역학',
    '무역전공':'무역학',
    '무역학부':'무역학',
    '화학과':'화학',
    '화학학과':'화학',
    '화학 전공':'화학',
    '공과대학 화학공학과':'화학공학',
    '화학 공학':'화학공학',
    '화학 공학과':'화학공학',
    '화학공':'화학공학',
    '화학공학과':'화학공학',
    '화학공항과':'화학공학',
    '화학공확과':'화학공학',
    '디자인학부 산업디자인과':'산업디자인학',
    '디자인학구 산업지자인전공':'산업디자인학',
    '산업 디자인':'산업디자인학',
    '산업디자인':'산업디자인학',
    '산업디자인 학과':'산업디자인학',
    '산업디자인과':'산업디자인학',
    '산업디자인부':'산업디자인학',
    '산업디자인하과':'산업디자인학',
    '산업디자인학과':'산업디자인학',
    '산업디자인학부':'산업디자인학',
    '의류학과':'의류학',
    '의류':'의류학',
    '의류헉과':'의류학',
    '디자인학 시각디자인전공':'시각디자인학',
    '비주얼 디자인':'시각디자인학',
    '비쥬얼디자인(시각)':'시각디자인학',
    '비쥬얼디자인(시각디자인)':'시각디자인학',
    '산업디자인 시각미디어전공':'시각디자인학',
    '산업디자인(시각)':'시각디자인학',
    '산업디자인(시각디자인 전공)':'시각디자인학',
    '산업디자인(시각디자인)':'시각디자인학',
    '산업디자인과 시각디자인전공':'시각디자인학',
    '산업디자인과시각디자인전공':'시각디자인학',
    '산업디자인학과 시각디자인전공':'시각디자인학',
    '산업디자인학과(시각디자인전공)':'시각디자인학',
    '산업미술학과 시각디자인':'시각디자인학',
    '시각 디자인':'시각디자인학',
    '시각디자인':'시각디자인학',
    '시각디자인 전공':'시각디자인학',
    '시각디자인과':'시각디자인학',
    '시각디자인학과':'시각디자인학',
    '중어중문':'중어중문학',
    '중어중문학과':'중어중문학',
    '중어중국학과':'중어중문학',
    '중어중문과':'중어중문학',
    '중어학과':'중어중문학',
    '중어중국':'중어중문학',
    '중어중국학':'중어중문학',
    '중어.중문학과':'중어중문학',
    '중어 중국학과':'중어중문학',
    '중국어학과':'중어중문학',
    '중국어과':'중어중문학',
    '중국어학':'중어중문학',
    '중국어중국학':'중어중문학',
    '중국어중국문학':'중어중문학',
    '중국어전공':'중어중문학',
    '중국어':'중어중문학',
    '중국어 학과':'중어중문학',
    '전자계산학과':'전자계산학',
    '전자계산':'전자계산학',
    '전자계산과':'전자계산학',
    '전자계산공학':'전자계산학',
    '전자계산학과(야)':'전자계산학',
    '전자계산공학과':'전자계산학',
    '컴퓨터공학(전자계산학)':'전자계산학',
    '일어일문':'일어일문학',
    '일어일문학과':'일어일문학',
    '일어일본과':'일어일문학',
    '일어일본학과':'일어일문학',
    '일어일문과':'일어일문학',
    '일어일본':'일어일문학',
    '일어':'일어일문학',
    '일어 일본학과':'일어일문학',
    '일어일문학전공':'일어일문학',
    '일어과':'일어일문학',
    '일어일문확과':'일어일문학',
    '일어일분학과':'일어일문학',
    '일어일본어':'일어일문학',
    '일어 일문학':'일어일문학',
    '관광 경영':'관광경영학',
    '관광경여':'관광경영학',
    '관광경영':'관광경영학',
    '관광경영학':'관광경영학',
    '관광경영학과':'관광경영학',
    '관광경영학과 경영학사':'관광경영학',
    '관광경영학전공':'관광경영학',
    '건축공학과':'건축공학',
    '건축공':'건축공학',
    '건축 공학과':'건축공학',
    '건축공학부':'건축공학',
    '패션디자인':'패션디자인학',
    '패션디자인학과':'패션디자인학',
    '패션디자인과':'패션디자인학',
    '패션디자인전공':'패션디자인학',
    '패션 디자인과':'패션디자인학',
    '의상섬유학부 패션디자인 전공':'패션디자인학',
    '패션디자인학부':'패션디자인학',
    '패션디자인 전공':'패션디자인학',
    '미술대학 패션디자인과':'패션디자인학',
    '패션디자인전공':'패션디자인학',
    '패션 디자인학과':'패션디자인학',
    '디자인학부 패션디자인과':'패션디자인학',
    '디자인대학 패션디자인학과':'패션디자인학',
    '미술대학 패션디자인학부':'패션디자인학',
    '패션지자인':'패션디자인학',
    '의상섬유학부 패션디자인':'패션디자인학',
    '패션디자인 2000.03~2008.02':'패션디자인학',
    '수학과':'수학',
    '수학학과':'수학',
    '수학전공':'수학',
    '전산통계':'전산통계학',
    '전산통계학과':'전산통계학',
    '전산통계학':'전산통계학',
    '전산 통계학':'전산통계학',
    '전산통게':'전산통계학',
    '전산통계학교':'전산통계학',
    '데이터정보(구 전산통계)':'전산통계학',
    '국어국문':'국어국문학',
    '국어국문학과':'국어국문학',
    '한국어문학':'국어국문학',
    '국어국문과':'국어국문학',
    '국어국문학가':'국어국문학',
    '인문학부 국어국문학과':'국어국문학',
    '국어국문학교':'국어국문학',
    '기계설계':'기계설계공학',
    '기계설계학과':'기계설계공학',
    '기계설계공학과':'기계설계공학',
    '기계설계학':'기계설계공학',
    '기게설계과':'기계설계공학',
    '기계공학부(기계설계)':'기계설계공학',
    '기계공학(기계설계)':'기계설계공학',
    '기계 설계':'기계설계공학',
    '기계설계 공학':'기계설계공학',
    '기계/설계':'기계설계공학',
    '기계설게공학과':'기계설계공학',
    '기계설계 공 학과':'기계설계공학',
    '신문방송':'신문방송학',
    '신문방송학과':'신문방송학',
    '신문방속학':'신문방송학',
    '언론정보학과(신문방송학과)':'신문방송학',
    '언론정보학과前신문방송학과':'신문방송학',
    '응용통계':'응용통계학',
    '응용통계학과':'응용통계학',
    '응용통계(경제학사)':'응용통계학',
    '상경 응용통계':'응용통계학',
    '상경대학 응용통계':'응용통계학',
    '응용통계학과(야간)':'응용통계학',
    '응용통계(상경)':'응용통계학',
    '경상대 응용통계':'응용통계학',
    '의상디자인':'의상디자인학',
    '의상디자인학과':'의상디자인학',
    '의상디자인과':'의상디자인학',
    '의생활학부 의상디자인':'의상디자인학',
    '의상 디자인':'의상디자인학',
    '의상 디자인과':'의상디자인학',
    '의상디자인전공':'의상디자인학',
    '의상 디자인학과':'의상디자인학',
    '공연예술학과 의상디자인':'의상디자인학',
    '의상디자인전공(대졸4)':'의상디자인학',
    '의상디자인학과졸':'의상디자인학',
    '사회학과':'사회학',
    '사회':'사회학',
    '토목공학과':'토목공학',
    '토목':'토목공학',
    '토목공':'토목공학',
    '토목공학부':'토목공학',
    '토목과':'토목공학',
    '토목학과':'토목공학',
    '토목공하과':'토목공학',
    '토목 공학과':'토목공학',
    '불어불문':'불어불문학',
    '불어불문학과':'불어불문학',
    '불어불문과':'불어불문학',
    '사학과':'사학',
    '역사학과':'사학',
    '역사학':'사학',
    '역사학전공':'사학',
    '인문학부 역사학과':'사학',
    '철학과':'철학',
    '철학전공':'철학',
    '의상학과':'의상학',
    '의상':'의상학',
    '의상학부':'의상학',
    '의상전공':'의상학',
    '물리학과':'물리학',
    '물리':'물리학',
    '물리&#54673':'물리학',
    '물리과':'물리학',
    '과학기술학부 물리학과':'물리학',
    '물리락과':'물리학',
    '일본어':'일어일문학',
    '일본어학과':'일어일문학',
    '일본어과':'일어일문학',
    '일본어학':'일어일문학',
    '일본어 학과':'일어일문학',
    '일본어 문학':'일어일문학',
    '일본어 전공':'일어일문학',
    '환경공학과':'환경공학',
    '환경학':'환경공학',
    '환경과학':'환경공학',
    '환경':'환경공학',
    '환경공':'환경공학',
    '환경학과':'환경공학',
    '환경과':'환경공학',
    '정치외교':'정치외교학',
    '정치외교학과':'정치외교학',
    '정치 외교 학과':'정치외교학',
    '정치 외교학과':'정치외교학',
    '독어독문':'독어독문학',
    '독어독문학과':'독어독문학',
    '독어학':'독어독문학',
    '독어 독문':'독어독문학',
    '독어독문 학과':'독어독문학',
    '독어독문과':'독어독문학',
    '생물학과':'생물학',
    '생물':'생물학',
    '생물과':'생물학',
    '사회복지':'사회복지학',
    '사회복지학과':'사회복지학',
    '사회복지과':'사회복지학',
    '시회복지학과':'사회복지학',
    '사회복지학부':'사회복지학',
    '금속공학과':'금속공학',
    '금속공':'금속공학',
    '금속':'금속공학',
    '금속공학교':'금속공학',
    '금속 공학과':'금속공학',
    '응용소재공학(구 금속공학)':'금속공학',
    '광고홍보':'광고홍보학',
    '광고홍보학과':'광고홍보학',
    '광고학과':'광고홍보학',
    '언론정보 광고홍보':'광고홍보학',
    '광고/홍보':'광고홍보학',
    '광고&PR':'광고홍보학',
    '광고홍':'광고홍보학',
    '광고,홍보':'광고홍보학',
    '경영학부 광고홍보':'광고홍보학',
    '광고학':'광고홍보학',
    '언론정보학부 광고홍보학 전공':'광고홍보학',
    '경영학부 광고.홍보학과':'광고홍보학',
    '광고':'광고홍보학',
    '체육학과':'체육학',
    '체육':'체육학',
    '체육과':'체육학',
    '체육대학':'체육학',
    '체육하과':'체육학',
    '체육학사':'체육학',
    '체육학전공 4학년':'체육학',
    '건강보건대학/체육학과':'체육학',
    '문헌정보':'문헌정보학',
    '문헌정보학과':'문헌정보학',
    '교육':'교육학',
    '전산':'전산학',
    '전산학과':'전산학',
    '전산과':'전산학',
    '전산학부':'전산학',
    '의류직물':'의류직물학',
    '의류직물학과':'의류직물학',
    '의류 직물학과':'의류직물학',
    '의류직물학전공':'의류직물학',
    '의류 직물':'의류직물학',
    '호텔관광경영':'호텔관광경영학',
    '호텔관광경영학과':'호텔관광경영학',
    '호텔관광 경영학과':'호텔관광경영학',
    '호텔관광경영하과':'호텔관광경영학',
    '호텔 관광 경영학과':'호텔관광경영학',
    '호텔.관광경영학':'호텔관광경영학',
    '호텔관광경영과':'호텔관광경영학',
    '호텔관관경영학과':'호텔관광경영학',
    '호텔관광':'호텔관광경영학',
    '호텔 관광경영학과':'호텔관광경영학',
    '호텔.관광 경영학과':'호텔관광경영학',
    '호텔관광경영대학':'호텔관광경영학',
    '식품영양':'식품영양학',
    '식품영양학과':'식품영양학',
    '심리학과':'심리학',
    '심리':'심리학',
    '식품공학과':'식품공학',
    '식품공':'식품공학',
    '식품':'식품공학',
    '재료공학과':'재료공학',
    '재료공':'재료공학',
    '재료':'재료공학',
    '재료 공학':'재료공학',
    '재료공합':'재료공학',
    '영어':'영어영문학',
    '영문학':'영어영문학',
    '영문학과':'영어영문학',
    '영문':'영어영문학',
    '영문과':'영어영문학',
    '영문학 전공':'영어영문학',
    '영여영문':'영어영문학',
    '영여영문학과':'영어영문학',
    '영여영문학':'영어영문학',
    '경영회계':'경영회계학',
    '경영회계학과':'경영회계학',
    '경영회계학부':'경영회계학',
    '경영/회계':'경영회계학',
    '경영.회계':'경영회계학',
    '경영.회계학과':'경영회계학',
    '경영,회계학':'경영회계학',
    '경영/회계학과':'경영회계학',
    '경영,회계학과':'경영회계학',
    '경영 회계':'경영회계학',
    '국제통상':'국제통상학',
    '국제통상학과':'국제통상학',
    '국제 통상':'국제통상학',
    '국제통상 학과':'국제통상학',
    '전자공':'전자공학',
    '통계학과':'통계학',
    '통계':'통계학',
    '경제학부 통계학':'통계학',
    '경제학부 통계학 전공':'통계학',
    '경제학부 통계학과':'통계학',
    '경제학부 통계학 전공':'통계학',
    '생명과학과':'생명과학',
    '생명과학부':'생명과학',
    '생명과학 전공':'생명과학',
    '생명과학학과':'생명과학',
    '생명과학전공':'생명과학',
    '도시공학과':'도시공학',
    '도시공':'도시공학',
    '도시공학전공':'도시공학',
    '제어계측':'제어계측학',
    '제어계측공학과':'제어계측학',
    '제어계측공':'제어계측학',
    '제어 계측':'제어계측학',
    '제어계측공학전공':'제어계측학',
    '제어계측공학 전공':'제어계측학',
    '사회체육':'사회체육학',
    '사회체육학과':'사회체육학',
    '사회체육과':'사회체육학',
    '사회체육 학과':'사회체육학',
    '호텔경영':'호텔경영학',
    '호텔경영학과':'호텔경영학',
    '호텔경영학':'호텔경영학',
    '호텔관광경영학부 호텔경영전공':'호텔경영학',
    '호텔경영전공':'호텔경영학',
    '경제무역':'경제무역학',
    '경제무역학과':'경제무역학',
    '경제/무역학':'경제무역학',
    '경제,무역':'경제무역학',
    '경제무역과':'경제무역학',
    '경제. 무역':'경제무역학',
    '경제.무역':'경제무역학',
    '가정관리':'가정관리학',
    '가정관리학과':'가정관리학',
    '가족환경(가정관리)':'가정관리학',
    '가정관리(현재아동가족학과로변경됨)':'가정관리학',
    '중국학과':'중국학',
    '중국':'중국학',
    '국제학부(중국학)':'중국학',
    '전자재료공학과':'전자재료공학',
    '전자재료':'전자재료공학',
    '전자재료공':'전자재료공학',
    '전자재료(전기전자공학부)':'전자재료공학',
    '전자재료 공학':'전자재료공학',
    '신학과':'신학',
    '신학부':'신학',
    '섬유공학과':'섬유공학',
    '의상섬유학부 섬유공학과':'섬유공학',
    '섬유 공학과':'섬유공학',
    '산업공':'산업공학',
    '유전공학과':'유전공학',
    '유전공':'유전공학',
    '생명과학 유전공':'유전공학',
    '멀티미디어':'멀티미디어학',
    '멀티미디어학과':'멀티미디어학',
    '멀티미디어학부':'멀티미디어학',
    '멀티미디어과':'멀티미디어학',
    '전산정보':'전산정보학',
    '정보전산':'전산정보학',
    '전산정보학과':'전산정보학',
    '정보전산학부':'전산정보학',
    '정보전산학':'전산정보학',
    '패션산업학과':'패션산업학',
    '의류패션산업':'패션산업학',
    '패션산업':'패션산업학',
    '패션산업학부':'패션산업학',
    '의류패션산업전공':'패션산업학',
    '의류패션산업학과':'패션산업학',
    '패션산업학':'패션산업학',
    '생활과학부 패션산업전공':'패션산업학',
    '패션산업전공':'패션산업학',
    '정보시스템':'정보시스템공학',
    '정보시스템학':'정보시스템공학',
    '정보시스템공학과':'정보시스템공학',
    '정보시스템 공학':'정보시스템공학',
    '건축':'건축학',
    '건축학과':'건축학',
    '건축과':'건축학',
    '신소재공학과':'신소재공학',
    '신소재':'신소재공학',
    '신소재학과':'신소재공학',
    '신소재공학':'신소재공학',
    '신소재 공학':'신소재공학',
    '신소재 공학과':'신소재공학',
    '신소재(고분재)공학과':'신소재공학',
    '신소재공학과(재료)':'신소재공학',
    '금속재료공학부 신소재공학':'신소재공학',
    '고분자공(신소재공학)':'신소재공학',
    '신소재공학부':'신소재공학',
    '신소재(고분자)공학':'신소재공학',
    '신소재과':'신소재공학',
    '자원공학과':'자원공학',
    '자원공':'자원공학',
    '공예학과':'공예학',
    '공예과':'공예학',
    '공예':'공예학',
    '가정학과':'가정학',
    '가정':'가정학',
    '가정학(인테리어)':'가정학',
    '가정학부':'가정학',
    '산업 경영학과':'산업경영공학',
    '산업경영':'산업경영공학',
    '산업경영공학과':'산업경영공학',
    '산업경영공학부':'산업경영공학',
    '산업경영학과':'산업경영공학',
    '전자전기공학부':'전기전자공학',
    '전자공학전공':'전자공학',
    '전자 공학':'전자공학',
    '전자공하과':'전자공학',
    '전자':'전자공학',
    '전자과':'전자공학',
    '전자전공':'전자공학',
    '전자 공학과':'전자공학'
})

In [75]:
pd.options.display.max_rows = 2300
df['대학전공'].value_counts()

경영학                                      2526
컴퓨터공학                                    1123
영어영문학                                    1056
경제학                                       915
전자공학                                      799
기계공학                                      780
무역학                                       755
회계학                                       632
산업공학                                      622
행정학                                       532
법학                                        513
중어중문학                                     500
일어일문학                                     460
전자계산학                                     417
화학공학                                      399
전기공학                                      387
산업디자인학                                    367
화학                                        339
국어국문학                                     324
정보통신학                                     323
의류학                                       299
건축공학                              

In [11]:
pd.options.display.max_rows = 1690
df['근무지역'].value_counts()

서울,경기,              3737
서울,,                3353
서울,서울,서울            3140
서울,경기,인천            1135
서울,경기,경기             875
서울,서울,경기             859
서울,인천,경기             687
서울,경기,해외             415
서울,경기,기타             390
서울,경기,부산             350
경기,서울,               340
서울,서울,               336
서울,경기,대전             322
서울,경기,서울             268
인천,서울,경기             239
서울,경기,강원             214
서울,경기,충남             189
서울,인천,               171
서울,경기,전북             155
서울,경기,광주             144
경기,서울,인천             143
서울,경기,미국             140
서울,부산,               139
서울,경기,전국             137
부산,경남,서울             134
부산,,                 134
부산,경남,울산             129
서울,경기,충북             128
경기,경기,경기             126
서울,경기,중국             124
서울,경기,일본             124
서울,해외,               123
경기,서울,경기             117
부산,경남,               113
부산,부산,부산             112
서울,부산,경기              95
경기,경기,서울              83
서울,경기,제주              82
인천,경기,서울              80
서울,미국,해외              74


In [12]:
df['근무지역']=df['근무지역'].replace(
{
    '서울,서울,서울':'서울',
    '서울,서울,':'서울',
    '서울,,':'서울',
    '서울,,서울':'서울',
    '부산,부산,부산':'부산',
    '부산,부산,':'부산',
    '부산,,':'부산',
    '부산,,부산':'부산',
    '경기,서울,':'서울,경기',
    '인천,서울,경기':'서울,경기,인천',
    '인천,경기,서울':'서울,경기,인천',
    '경기,인천,서울':'서울,경기,인천',
    '경기,서울,인천':'서울,경기,인천',
    '서울,경기,인천':'서울,경기,인천',
    '서울,인천,경기':'서울,경기,인천',
    '경기,서울,경기':'경기,경기,서울',
    '경기,경기,서울':'경기,경기,서울',
    '서울,서울,경기':'경기,서울,서울',
    '서울,경기,서울':'경기,서울,서울',
    '서울,경기,해외':'서울,경기,해외',
    '서울,해외,경기':'서울,경기,해외',
    '해외,서울,경기':'서울,경기,해외',
    '해외,경기,서울':'서울,경기,해외',
    '경기,해외,서울':'서울,경기,해외',
    '경기,서울,해외':'서울,경기,해외',
    '서울,경기,기타':'서울,경기,기타',
    '서울,기타,경기':'서울,경기,기타',
    '경기,서울,기타':'서울,경기,기타',
    '경기,기타,서울':'서울,경기,기타',
    '기타,서울,경기':'서울,경기,기타',
    '기타,경기,서울':'서울,경기,기타',
    '부산,서울,경기':'서울,경기,부산',
    '부산,경기,서울':'서울,경기,부산',
    '경기,부산,서울':'서울,경기,부산',
    '경기,서울,부산':'서울,경기,부산',
    '서울,경기,부산':'서울,경기,부산',
    '서울,부산,경기':'서울,경기,부산',
    
    
    
    
})                         

In [13]:
df['직무태그']

0                          취재기자, 편집 기사, 유아 사이트 기획, 시나리오 작성
1                                                      NaN
2        하드웨어 , 무선통신 , MICOM , ASM , RF , CDMA , Firm W...
3                                                      반도체
4                                                      기술직
                               ...                        
11043                                             제품,산업,팬시
11044                                                물류,유통
11045                                                  NaN
11046                                                   판매
11047                                                  NaN
Name: 직무태그, Length: 27618, dtype: object